In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('fso20-21data.txt')
winter2 = pd.read_csv ('fso21-22data.txt')
winter3 = pd.read_csv ('fso22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
3,FSO,2021-12-01 01:15:00,0.0,-SN
4,FSO,2021-12-01 01:35:00,0.0,-SN
5,FSO,2021-12-01 01:55:00,0.0,-SN
6,FSO,2021-12-01 02:15:00,0.0,-SN
7,FSO,2021-12-01 02:35:00,0.0,-SN
...,...,...,...,...
6306,FSO,2022-02-27 16:15:00,0.0,-SN
6317,FSO,2022-02-27 19:55:00,0.0,-SN
6319,FSO,2022-02-27 20:35:00,0.0,-SN
6320,FSO,2022-02-27 20:55:00,0.0,-SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 55]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 55]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 55]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
fso = LocationInfo(
    'Highgate', 'Vermont', 
    latitude=44.9565,
    longitude=-73.0511, 
    timezone='America/New_York'
)

fso

LocationInfo(name='Highgate', region='Vermont', timezone='America/New_York', latitude=44.9565, longitude=-73.0511)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(fso.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(fso.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(fso.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:10:54  2020-12-01 16:11:47  2020-12-01
1   2020-12-02 07:12:02  2020-12-02 16:11:27  2020-12-02
2   2020-12-03 07:13:08  2020-12-03 16:11:10  2020-12-03
3   2020-12-04 07:14:12  2020-12-04 16:10:55  2020-12-04
4   2020-12-05 07:15:15  2020-12-05 16:10:43  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:38:21  2021-02-24 17:33:00  2021-02-24
86  2021-02-25 06:36:39  2021-02-25 17:34:22  2021-02-25
87  2021-02-26 06:34:57  2021-02-26 17:35:44  2021-02-26
88  2021-02-27 06:33:14  2021-02-27 17:37:06  2021-02-27
89  2021-02-28 06:31:30  2021-02-28 17:38:28  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:10:38  2021-12-01 16:11:51  2021-12-01
1   2021-12-02 07:11:46  2021-12-02 16:11:31  2021-12-02
2   2021-12-03 07:12:52  2021-12-03 16:11:13  2021-12-03
3   2021-12-04 07:13:56  2021-12-04 16:10:58  2021-12-04
4   2021-12-05 07:15:00  2021-12-05 16:10:45  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:38:45  2022-02-24 17:32:40  2022-02-24
86  2022-02-25 06:37:04  2022-02-25 17:34:02  2022-02-25
87  2022-02-26 06:35:22  2022-02-26 17:35:24  2022-02-26
88  2022-02-27 06:33:39  2022-02-27 17:36:46  2022-02-27
89  2022-02-28 06:31:55  2022-02-28 17:38:08  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:10:22  2022-12-01 16:11:56  2022-12-01
1   2022-12-02 07:11:29  2022-12-02 16:11:35  2022-12-02
2   2022-12-03 07:12:36  2022-12-03 16:11:17  2022-12-03
3   2022-12-04 07:13:41  2022-12-04 16:11:01  2022-12-04
4   2022-12-05 07:14:45  2022-12-05 16:10:48  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:39:10  2023-02-24 17:32:20  2023-02-24
86  2023-02-25 06:37:29  2023-02-25 17:33:42  2023-02-25
87  2023-02-26 06:35:47  2023-02-26 17:35:04  2023-02-26
88  2023-02-27 06:34:04  2023-02-27 17:36:27  2023-02-27
89  2023-02-28 06:32:21  2023-02-28 17:37:48  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FSO,2020-12-01 03:55:00,0.0,-RA,2020-12-01,2020-12-01 07:10:54,2020-12-01 16:11:47
1,FSO,2020-12-01 04:55:00,0.0,-RA,2020-12-01,2020-12-01 07:10:54,2020-12-01 16:11:47
2,FSO,2020-12-01 05:55:00,0.0,RA,2020-12-01,2020-12-01 07:10:54,2020-12-01 16:11:47
3,FSO,2020-12-01 06:55:00,0.0,+RA,2020-12-01,2020-12-01 07:10:54,2020-12-01 16:11:47
4,FSO,2020-12-01 18:55:00,0.0,-RA,2020-12-01,2020-12-01 07:10:54,2020-12-01 16:11:47
...,...,...,...,...,...,...,...
313,FSO,2021-02-25 13:55:00,0.0,-SN,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22
314,FSO,2021-02-25 14:55:00,0.0,-SN,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22
315,FSO,2021-02-27 18:55:00,0.0,UP,2021-02-27,2021-02-27 06:33:14,2021-02-27 17:37:06
316,FSO,2021-02-27 21:55:00,0.0,-RA,2021-02-27,2021-02-27 06:33:14,2021-02-27 17:37:06


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FSO,2021-12-01 01:55:00,0.0,-SN,2021-12-01,2021-12-01 07:10:38,2021-12-01 16:11:51
1,FSO,2021-12-01 02:55:00,0.0,-SN,2021-12-01,2021-12-01 07:10:38,2021-12-01 16:11:51
2,FSO,2021-12-01 03:55:00,0.0,-SN,2021-12-01,2021-12-01 07:10:38,2021-12-01 16:11:51
3,FSO,2021-12-01 04:55:00,0.0,-SN,2021-12-01,2021-12-01 07:10:38,2021-12-01 16:11:51
4,FSO,2021-12-01 05:55:00,0.0,UP,2021-12-01,2021-12-01 07:10:38,2021-12-01 16:11:51
...,...,...,...,...,...,...,...
325,FSO,2022-02-26 17:55:00,0.0,-SN,2022-02-26,2022-02-26 06:35:22,2022-02-26 17:35:24
326,FSO,2022-02-27 06:55:00,0.0,-SN,2022-02-27,2022-02-27 06:33:39,2022-02-27 17:36:46
327,FSO,2022-02-27 15:55:00,0.0,-SN,2022-02-27,2022-02-27 06:33:39,2022-02-27 17:36:46
328,FSO,2022-02-27 19:55:00,0.0,-SN,2022-02-27,2022-02-27 06:33:39,2022-02-27 17:36:46


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FSO,2022-12-01 15:55:00,0.0,-SN,2022-12-01,2022-12-01 07:10:22,2022-12-01 16:11:56
1,FSO,2022-12-01 16:55:00,0.0,-SN,2022-12-01,2022-12-01 07:10:22,2022-12-01 16:11:56
2,FSO,2022-12-03 02:55:00,0.0,-RA,2022-12-03,2022-12-03 07:12:36,2022-12-03 16:11:17
3,FSO,2022-12-03 13:55:00,0.0,RA,2022-12-03,2022-12-03 07:12:36,2022-12-03 16:11:17
4,FSO,2022-12-03 14:55:00,0.0,RA,2022-12-03,2022-12-03 07:12:36,2022-12-03 16:11:17
...,...,...,...,...,...,...,...
367,FSO,2023-02-27 03:55:00,0.0,-SN,2023-02-27,2023-02-27 06:34:04,2023-02-27 17:36:27
368,FSO,2023-02-27 04:55:00,0.0,-SN,2023-02-27,2023-02-27 06:34:04,2023-02-27 17:36:27
369,FSO,2023-02-27 05:55:00,0.0,-SN,2023-02-27,2023-02-27 06:34:04,2023-02-27 17:36:27
370,FSO,2023-02-27 06:55:00,0.0,-SN,2023-02-27,2023-02-27 06:34:04,2023-02-27 17:36:27


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
15,FSO,2020-12-08 09:55:00,0.0,-SN,2020-12-08,2020-12-08 07:18:14,2020-12-08 16:10:23
16,FSO,2020-12-08 10:55:00,0.0,-SN,2020-12-08,2020-12-08 07:18:14,2020-12-08 16:10:23
17,FSO,2020-12-08 12:55:00,0.0,-SN,2020-12-08,2020-12-08 07:18:14,2020-12-08 16:10:23
18,FSO,2020-12-08 13:55:00,0.0,-SN,2020-12-08,2020-12-08 07:18:14,2020-12-08 16:10:23
19,FSO,2020-12-08 14:55:00,0.0,-SN,2020-12-08,2020-12-08 07:18:14,2020-12-08 16:10:23
...,...,...,...,...,...,...,...
310,FSO,2021-02-25 07:55:00,0.0,-SN,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22
311,FSO,2021-02-25 08:55:00,0.0,SN,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22
312,FSO,2021-02-25 09:55:00,0.0,-SN,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22
313,FSO,2021-02-25 13:55:00,0.0,-SN,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
15,FSO,2020-12-08 09:55:00,0.0,-SN,2020-12-08,2020-12-08 07:18:14,2020-12-08 16:10:23,2020-12-08 08:18:14,2020-12-08 15:10:23
16,FSO,2020-12-08 10:55:00,0.0,-SN,2020-12-08,2020-12-08 07:18:14,2020-12-08 16:10:23,2020-12-08 08:18:14,2020-12-08 15:10:23
17,FSO,2020-12-08 12:55:00,0.0,-SN,2020-12-08,2020-12-08 07:18:14,2020-12-08 16:10:23,2020-12-08 08:18:14,2020-12-08 15:10:23
18,FSO,2020-12-08 13:55:00,0.0,-SN,2020-12-08,2020-12-08 07:18:14,2020-12-08 16:10:23,2020-12-08 08:18:14,2020-12-08 15:10:23
19,FSO,2020-12-08 14:55:00,0.0,-SN,2020-12-08,2020-12-08 07:18:14,2020-12-08 16:10:23,2020-12-08 08:18:14,2020-12-08 15:10:23
...,...,...,...,...,...,...,...,...,...
310,FSO,2021-02-25 07:55:00,0.0,-SN,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22,2021-02-25 07:36:39,2021-02-25 16:34:22
311,FSO,2021-02-25 08:55:00,0.0,SN,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22,2021-02-25 07:36:39,2021-02-25 16:34:22
312,FSO,2021-02-25 09:55:00,0.0,-SN,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22,2021-02-25 07:36:39,2021-02-25 16:34:22
313,FSO,2021-02-25 13:55:00,0.0,-SN,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22,2021-02-25 07:36:39,2021-02-25 16:34:22


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.0
2021-2022: 0.0
2022-2023: 0.0


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FSO,2020-12-01 03:55:00,0.0,-RA,2020-12-01,2020-12-01 07:10:54,2020-12-01 16:11:47
1,FSO,2020-12-01 04:55:00,0.0,-RA,2020-12-01,2020-12-01 07:10:54,2020-12-01 16:11:47
2,FSO,2020-12-01 05:55:00,0.0,RA,2020-12-01,2020-12-01 07:10:54,2020-12-01 16:11:47
3,FSO,2020-12-01 06:55:00,0.0,+RA,2020-12-01,2020-12-01 07:10:54,2020-12-01 16:11:47
4,FSO,2020-12-01 18:55:00,0.0,-RA,2020-12-01,2020-12-01 07:10:54,2020-12-01 16:11:47
...,...,...,...,...,...,...,...
307,FSO,2021-02-25 02:55:00,0.0,-RA,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22
308,FSO,2021-02-25 05:55:00,0.0,-RA,2021-02-25,2021-02-25 06:36:39,2021-02-25 17:34:22
315,FSO,2021-02-27 18:55:00,0.0,UP,2021-02-27,2021-02-27 06:33:14,2021-02-27 17:37:06
316,FSO,2021-02-27 21:55:00,0.0,-RA,2021-02-27,2021-02-27 06:33:14,2021-02-27 17:37:06


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31012\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.0
2021-2022: 0.0
2022-2023: 0.0
